# TSFM Client - Simple Demo

This notebook demonstrates the simplest way to use the TSFM (Time Series Foundation Model) client for forecasting.

## Prerequisites

1. **API Key**: You need to set the `TSFM_API_KEY` environment variable
2. **Server**: Make sure the TSFM server is running on `http://localhost:8000`

### Setting up the API Key

```bash
export TSFM_API_KEY=""
```

Or in Python:
```python
import os
os.environ['TSFM_API_KEY'] = ''
```

In [1]:
import os
os.environ['TSFM_API_KEY'] = ''

In [2]:
# Set up API key (do this once per notebook session)
import os
# os.environ['TSFM_API_KEY'] = ''

# Install the package if needed
# !pip install tsfm-client

# Import required libraries
import pandas as pd
import numpy as np
from tsfm_client import predict

print("✅ Setup complete!")

✅ Setup complete!


In [18]:
# Create sample sales data
n = 5000
dates = pd.date_range('2024-01-01', periods=n, freq='D')

# Simulate sales data with trend and noise
trend = np.linspace(100, 200, n)
noise = np.random.normal(0, 10, n)
seasonal = 10 * np.sin(2 * np.pi * np.arange(n) / 7)  # Weekly seasonality

sales_data = pd.Series(trend + seasonal + noise, index=dates, name='sales')

print("Sample sales data (last 10 days):")
print(sales_data.tail(10))
sales_data.shape

Sample sales data (last 10 days):
2037-08-30    187.615498
2037-08-31    202.484798
2037-09-01    204.683764
2037-09-02    209.230129
2037-09-03    210.950755
2037-09-04    189.161304
2037-09-05    186.971624
2037-09-06    215.754153
2037-09-07    208.011925
2037-09-08    213.200960
Freq: D, Name: sales, dtype: float64


(5000,)

In [19]:
# Make prediction with confidence intervals
response = predict(
    data=sales_data,
    forecast_horizon=10,
    confidence_intervals=True,
    model='toto-open-base-1.0',
    num_samples=10
)

print(f"✅ 10-day forecast: {response.forecast}")
print(f"Model used: {response.model_name}")

if response.confidence_intervals:
    print(f"\nConfidence intervals: {response.confidence_intervals}")

✅ 10-day forecast: [207.3481903076172, 202.52542114257812, 193.8355712890625, 196.54800415039062, 196.25160217285156, 201.46533203125, 205.7838897705078, 210.92056274414062, 201.18551635742188, 191.5183868408203]
Model used: toto-open-base-1.0

Confidence intervals: {'80%': {'lower': [200.07669067382812, 188.9459991455078, 186.9412078857422, 188.52622985839844, 185.835205078125, 191.2191619873047, 190.85610961914062, 205.53067016601562, 188.56097412109375, 182.3580780029297], 'upper': [213.99501037597656, 215.4633026123047, 200.60580444335938, 206.3490753173828, 207.14427185058594, 219.1131591796875, 217.5805206298828, 220.9561767578125, 211.22903442382812, 199.36155700683594]}}


In [6]:
# Convert to DataFrame for easier analysis
forecast_df = response.to_pandas()
print("Forecast DataFrame:")
print(forecast_df)

Forecast DataFrame:
     forecast   lower_80%   upper_80%
0  207.859421  197.372620  219.670868
1  205.651657  192.957123  223.644821
2  207.749023  188.100082  228.943390
3  204.989349  184.788467  222.761703
4  203.002380  192.074020  215.586548
5  202.340042  188.210464  211.060654
6  207.307465  186.002716  229.053802
7  206.755539  189.093567  227.177200
8  211.833374  180.924911  227.729141
9  213.599564  183.684586  230.157669


In [7]:
# Optional: Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Plot historical data
plt.plot(sales_data.index, sales_data.values, label='Historical Sales', color='blue')

# Plot forecast
forecast_dates = pd.date_range(sales_data.index[-1] + pd.Timedelta(days=1), periods=10, freq='D')
plt.plot(forecast_dates, response.forecast, label='Forecast', color='red', linestyle='--')

# Plot confidence intervals if available
if response.confidence_intervals:
    ci_80 = response.confidence_intervals['80%']
    plt.fill_between(forecast_dates, ci_80['lower'], ci_80['upper'], 
                    alpha=0.3, color='red', label='80% Confidence Interval')

plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Sales Forecast with TSFM')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'